# Usar um Amazon SageMaker Studio Notebook

Este notebook percorre o processo de inicialização e configuração do ambiente. Ele mostra como carregar e baixar os dados de/para um bucket do Amazon Simple Storage Service (Amazon S3) usando os métodos auxiliares do Amazon S3 por meio do SDK Python do SageMaker.

Comece inicializando o ambiente. Para isso, importe as bibliotecas necessárias e obtenha o bucket padrão do Amazon S3 que o SageMaker Studio usa.

In [ ]:
# Install dependencies
import pandas as pd
import boto3


# SageMaker dependencies
import sagemaker
from sagemaker import get_execution_role, session
from sagemaker.s3 import S3Downloader, S3Uploader

region= boto3.Session().region_name

# This object represents the AWS Identity and Access Management (IAM) role that you are assigned
role = sagemaker.get_execution_role()
print("Role: ", role)

sm_session = session.Session(boto3.Session())
sm = boto3.Session().client("sagemaker")
sm_runtime = boto3.Session().client("sagemaker-runtime")

Baixe o conjunto de dados do bucket do Amazon S3 usando o método S3Downloader do SDK Python do SageMaker. Consulte [método S3Downloader do SDK Python do SageMaker](https://sagemaker.readthedocs.io/en/stable/api/utility/s3.html#sagemaker.s3.S3Downloader) para saber mais sobre esse método de download.

In [ ]:
bucket = ''
s3 = boto3.resource('s3')
for buckets in s3.buckets.all():
    if 'labdatabucket' in buckets.name:
        bucket = buckets.name
print(bucket)
prefix = 'scripts/data'

S3Downloader.download(s3_uri=f"s3://{bucket}/{prefix}/iris.csv", local_path= 'data/')

O código a seguir verifica o conjunto de dados e exibe-o em uma grade.

In [ ]:
import pandas as pd
import numpy as np

shape=pd.read_csv("data/iris.csv", header=None)
shape.sample(3)

Particione o conjunto de dados em divisões de treinamento e teste. Carregue-o no bucket do Amazon S3 usando o método S3Uploader do SDK Python do SageMaker. Consulte [método S3Uploader do SDK Python do SageMaker](https://sagemaker.readthedocs.io/en/stable/api/utility/s3.html#sagemaker.s3.S3Uploader) para saber mais sobre esse método de upload.

In [ ]:
train_data = shape.sample(frac=0.8,random_state=200)
test_data = shape.drop(train_data.index)

In [ ]:
train_file = 'train_data.csv';
train_data.to_csv(train_file, index=False, header=True)
test_file = 'test_data.csv';
test_data.to_csv(test_file, index=False, header=True)

# Return the URLs of the uploaded file, so they can be reviewed or used elsewhere
s3url = S3Uploader.upload(train_file, 's3://{}/{}'.format(bucket, prefix + "/train", "train"))
print(s3url)
s3url = S3Uploader.upload(test_file, 's3://{}/{}'.format(bucket, prefix + "/test", "test"))
print(s3url)

### Conclusão

Parabéns! Você iniciou o ambiente e carregou e baixou arquivos usando um bucket do Amazon S3 com sucesso. 

## Limpeza

Você concluiu este notebook. Passe para a próxima parte do laboratório da seguinte forma:

- Feche este arquivo de notebook
- Retorne às instruções do laboratório